In [1]:
import torch
from torch import nn
from torch.nn import functional as F

# 生成一个网络，其中包含两个全连接层和一个ReLU激活函数
# 输入维度为20，隐藏单元数为256，输出维度为10
# 第一层的输入是20维的，输出是256维的
# 第二层的输入是256维的，输出是10维的
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.2199,  0.1636,  0.2522, -0.0508, -0.1539,  0.4361,  0.1147,  0.0307,
          0.2733, -0.2589],
        [ 0.2802,  0.1118,  0.3102,  0.0556, -0.1625,  0.2453,  0.0638,  0.2020,
          0.2572, -0.1137]], grad_fn=<AddmmBackward0>)

In [2]:
# 自定义一个多层感知机类，功能和上面的网络相同
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.0308,  0.1629,  0.1283,  0.0202,  0.3361, -0.0224, -0.1319, -0.0071,
          0.0989,  0.1508],
        [ 0.0344, -0.0412,  0.0772, -0.0352,  0.3577,  0.0302, -0.0370,  0.0525,
          0.1395,  0.1890]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2181,  0.1488, -0.2267,  0.0718, -0.0511, -0.1958,  0.1572, -0.1381,
          0.0713, -0.0148],
        [ 0.1200,  0.0749, -0.2058,  0.0850, -0.1138, -0.1358,  0.1273, -0.2110,
         -0.0654,  0.0693]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(-0.3839, grad_fn=<SumBackward0>)

In [8]:
# 展示一下嵌套的网络结构
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2973, grad_fn=<SumBackward0>)